## Set **seed**

In [ ]:
from sefixlines.utils import set_all_seeds

set_all_seeds()

## Data

In [ ]:
from sefixlines.datasets import TextClassificationDataset

### **Initial**

In [ ]:
texts = []
labels = []

In [ ]:
classes = []

### **Split**

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, valid_texts, train_labels, valid_labels = train_test_split(texts, labels, test_size=0.2, random_state=42) # stratify=labels

### Create **Datasets**

In [ ]:
dataset = TextClassificationDataset(texts, labels)

train_set = TextClassificationDataset(train_texts, train_labels)
valid_set = TextClassificationDataset(valid_texts, valid_labels)

### ***Visualization***

In [ ]:
dataset.show(classes=classes)

## **Models**

In [ ]:
from torch import nn, optim
from sefixlines.models import Classifier

### *Score*

In [ ]:
scores = dict()

### **Model**: `papluca/xlm-roberta-base-language-detection`

In [ ]:
model_id = 'papluca/xlm-roberta-base-language-detection'

In [ ]:
from transformers import AutoTokenizer

TextClassificationDataset.max_length = 128
TextClassificationDataset.tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
from sefixlines.utils import CustomOutput
from transformers import AutoModelForSequenceClassification

model = CustomOutput(
    AutoModelForSequenceClassification.from_pretrained(
        model_id, 
        num_labels=len(classes), 
        ignore_mismatched_sizes=True
    )
)
optimizer = optim.Adam(model.parameters(), lr=5e-5)

In [ ]:
model_wrapped = Classifier(model, model_id.split('/')[-1], optimizer=optimizer)
model_wrapped.fit(train_set, valid_set, num_epochs=3)

In [ ]:
scores[model_wrapped.best_score] = model_wrapped
model_wrapped.visualize_predictions(valid_set, classes=classes)

## Result

In [ ]:
best_model_wrapped = scores[max(scores)]
best_model_wrapped.name

## Submission

In [ ]:
test_texts = []
test_set = TextClassificationDataset(test_texts)

In [ ]:
prediction_class_id = best_model_wrapped.predict(test_set)
prediction_class_names = [classes[class_id] for class_id in prediction_class_id]